In [1]:
# git clone package:
# !git clone <path_package>
# or copy package :
!cp -r /kaggle/input/autonlp-git/SentimentML-main /kaggle/working

In [2]:
import os
# directory of package :
fd = os.open("/kaggle/working/SentimentML-main", os.O_RDONLY )
os.fchdir(fd)
os.getcwd()

'/kaggle/working/SentimentML-main'

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 561 kB 287 kB/s 
     |████████████████████████████████| 7.7 MB 6.2 MB/s 
     |████████████████████████████████| 280 kB 67.0 MB/s 
     |████████████████████████████████| 11.6 MB 67.2 MB/s 
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 80 kB 6.8 MB/s 
     |████████████████████████████████| 23.9 MB 1.0 MB/s 
     |████████████████████████████████| 2.6 MB 71.1 MB/s 
     |████████████████████████████████| 283 kB 65.5 MB/s 
     |████████████████████████████████| 70 kB 7.9 MB/s 
     |████████████████████████████████| 10.8 MB 66.5 MB/s 
     |████████████████████████████████| 301 kB 53.6 MB/s 
     |████████████████████████████████| 166.7 MB 16 kB/s 
     |████████████████████████████████| 46 kB 2.6 MB/s 
     |████████████████████████████████| 12.8 MB 39.4 MB/s 
     |████████████████████████████████| 394.3 MB 11 kB/s 
     |████████████████████████████████| 47.5 MB 38 kB/s 
     |████████████████

In [4]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-2.0.0-py3-none-any.whl
     |████████████████████████████████| 2.9 MB 168 kB/s 
  Using cached huggingface_hub-0.0.17-py3-none-any.whl (52 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.3.0 requires transformers<4.1,>=4.0, but you have transformers 4.11.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
# if png doesn't save correctly
!pip install kaleido

     |████████████████████████████████| 79.9 MB 153 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
# if spacy can't load spacy model :
!python3 -m spacy download fr_core_news_md

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
     |████████████████████████████████| 46.1 MB 4.1 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.3.0-py3-none-any.whl size=46100669 sha256=601c4eee68abc7aa7a7d092306b438334ccb957fa2fdad88998d947ca955df0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hb42a5rk/wheels/62/a5/93/9c82d8ae84430ed80720262c11726d046d3444af301b7d2895
Successfully built fr-core-news-md
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


# Import

In [7]:
import pandas as pd
from autonlp.autonlp import AutoNLP
import os
from pathlib import Path
from autonlp.flags import Flags, save_yaml
import dataclasses

# Flags update (parameters)

In [8]:
# add logs from optimization to outdir :
!cp -r /kaggle/input/logs-validation/logs /kaggle/working/logs

In [9]:
#####################
# Parameters
#####################

flags_dict_info = {
    "path_data": "/kaggle/input/finance-corpus/FinancialPhraseBank_train.csv",
    "path_data_validation": "",
    "apply_logs": True,
    "outdir": "/kaggle/working/logs",
    "seed": 15,
    "debug": False,  # for debug : use only 50 data rows for training
}
flags_dict_preprocessing = {
    "column_text": "text_fr",  # name column with texts
    "target": "sentiment",     # name column with targets
    "language_text": "fr",
    "apply_small_clean": True,
    "name_spacy_model": "fr_core_news_md",
    "apply_spacy_preprocessing": True,
    "apply_entity_preprocessing": True
}

flags_dict_autonlp = {
    "objective": 'multi-class',    # 'binary' or 'multi-class' or 'regression'
    
    "embedding": {"tf": 1, "tf-idf": 2, "word2vec": None, "fasttext": None, "doc2vec": None, "transformer": None},
    
    "classifier": {"Naive_Bayes": [1], "Logistic_Regression": [1], "SGD_Classifier": [2],
                   "XGBoost": [], "Global_Average": [], "Attention": [], "BiRNN": [], "BiRNN_Attention": [],
                   "biLSTM": [], "BiLSTM_Attention": [], "biGRU": [], "BiGRU_Attention": []},
    
    "max_run_time_per_model": 60,
    "frac_trainset": 0.7,
    "scoring": 'f1',
    "average_scoring": "macro",
    "nfolds": 5,
    "nfolds_train": 5,
    "class_weight": True,
    "apply_blend_model": True,
    "verbose": 2,
    "method_embedding": {'Word2vec': 'Word2Vec',
                         'Fasttext': 'FastText',
                         'Doc2Vec': 'Doc2Vec',
                         'Transformer': 'CamemBERT',
                         'spacy': [(['ADJ', 'NOUN', 'VERB'], True)]},

    "apply_optimization": False,
    "apply_validation": True,
    
    "batch_size":32,
    "tr_learning_rate": [3e-5],
    "tr_maxlen": 100,
    "patience":2
}

flags_dict_display = {
    "sort_leaderboard": 'f1'
}

flags = Flags().update(flags_dict_info)
flags = flags.update(flags_dict_preprocessing)
flags = flags.update(flags_dict_autonlp)
flags = flags.update(flags_dict_display)
print("flags :", flags)
debug = flags.debug
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)
flags_dict = dataclasses.asdict(flags)
save_yaml(outdir / "flags.yaml", flags_dict)

flags : Flags(path_data='/kaggle/input/finance-corpus/FinancialPhraseBank_train.csv', path_data_validation='', apply_logs=True, outdir='/kaggle/working/logs', apply_mlflow=False, experiment_name='Experiment', apply_app=False, debug=False, seed=15, column_text='text_fr', language_text='fr', target='sentiment', apply_small_clean=True, name_spacy_model='fr_core_news_md', apply_spacy_preprocessing=True, apply_entity_preprocessing=True, objective='multi-class', embedding={'tf': 1, 'tf-idf': 2, 'word2vec': None, 'fasttext': None, 'doc2vec': None, 'transformer': None}, classifier={'Naive_Bayes': [1], 'Logistic_Regression': [1], 'SGD_Classifier': [2], 'XGBoost': [], 'Global_Average': [], 'Attention': [], 'BiRNN': [], 'BiRNN_Attention': [], 'biLSTM': [], 'BiLSTM_Attention': [], 'biGRU': [], 'BiGRU_Attention': []}, regressor={'SGD_Regressor': [1], 'XGBoost': [], 'Global_Average': [2], 'Attention': [], 'BiRNN': [], 'BiRNN_Attention': [], 'biLSTM': [], 'BiLSTM_Attention': [], 'biGRU': [], 'BiGRU_A

# Instantiation

In [10]:
autonlp = AutoNLP(flags)

# Preprocessing

In [11]:
%%time
autonlp.data_preprocessing()


Read data...

Begin preparation of 3876 data :

Training set size : 2713
Test set size : 1163

Begin preprocessing of 2713 train data :
⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
- Apply small clean of texts...
- Apply nlp.pipe from spacy...
- Apply entities preprocessing...

Begin preprocessing of 1163 test data :
- Apply small clean of texts...
- Apply nlp.pipe from spacy...
- Apply entities preprocessing...
CPU times: user 19.9 s, sys: 1.67 s, total: 21.5 s
Wall time: 26.9 s


# Training

In [12]:
#autonlp.train()

# Leaderboard (Train score)

In [13]:
#leaderboard_train = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='train')
#print('Train Leaderboard')
#leaderboard_train.to_csv(os.path.join(flags.outdir,'leaderboard_train.csv'), index=False)
#leaderboard_train

# Leaderboard (Validation score)

In [14]:
#leaderboard_val = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='val')
#print('\nValidation Leaderboard')
#leaderboard_val.to_csv(os.path.join(flags.outdir,'leaderboard_val.csv'), index=False)
#leaderboard_val

In [15]:
# correlation between oof_val predictions
#autonlp.correlation_models()

In [16]:
#df_all_results = pd.read_csv("/kaggle/working/logs/df_all_results.csv")
#if len(df_all_results) > 0:
#    df_all_results_mean = df_all_results.groupby('model').mean().sort_values('mean_test_score', ascending=False)
#    print('\nGridSearch information Leaderboard')
#    df_all_results.to_csv(os.path.join(flags.outdir,'df_all_results_mean.csv'), index=False)
#    print(df_all_results_mean)

In [17]:
#df_all_results

In [18]:
#df_all_results.groupby(['model'])['mean_test_score'].max()

In [19]:
#from PIL import Image
#Image.open(os.path.join(flags.outdir, 'boxplot_df_all_results.png'))

In [20]:
#try:
#    Image.open(os.path.join(flags.outdir, 'last_logs', 'metric_scores_val.png'))
#except:
#    pass

# Testing on test set from train/test split

In [21]:
name_logs = 'best_logs'
on_test_data = True
autonlp.leader_predict(name_logs = name_logs, on_test_data = on_test_data)


tf+Logistic_Regression_ADJ_NOUN_VERB_lem Model:
Model fold2.joblib:
Inference : 1163 samples in 0.03335428237915039 sec ( 34868.08640580995 / sample).

Scores :
accuracy = 0.7446
precision macro = 0.7017
recall macro = 0.6526
f1 score macro = 0.672


Model fold3.joblib:
Inference : 1163 samples in 0.03677535057067871 sec ( 31624.44360019968 / sample).

Scores :
accuracy = 0.7291
precision macro = 0.6835
recall macro = 0.6285
f1 score macro = 0.6501


Model fold0.joblib:
Inference : 1163 samples in 0.03709578514099121 sec ( 31351.27065190146 / sample).

Scores :
accuracy = 0.7549
precision macro = 0.7116
recall macro = 0.6514
f1 score macro = 0.6748


Model fold4.joblib:
Inference : 1163 samples in 0.03315329551696777 sec ( 35079.468929560244 / sample).

Scores :
accuracy = 0.7592
precision macro = 0.7183
recall macro = 0.677
f1 score macro = 0.6946


Model fold1.joblib:
Inference : 1163 samples in 0.032527923583984375 sec ( 35753.89609475783 / sample).

Scores :
accuracy = 0.7524
prec

In [22]:
leaderboard_test = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='test', info_models=autonlp.info_models)
print('\nTest Leaderboard')
leaderboard_test


Test Leaderboard


,name,accuracy_test,recall_macro_test,precision_macro_test,f1_macro_test
3,BlendModel,0.7610,0.6768,0.7255,0.6968
2,tf-idf+SGD_Classifier_ADJ_NOUN_VERB_lem,0.7567,0.6722,0.7217,0.6928
0,tf+Logistic_Regression_ADJ_NOUN_VERB_lem,0.7575,0.6598,0.7237,0.6848
1,tf+Naive_Bayes_ADJ_NOUN_VERB_lem,0.7102,0.6393,0.6579,0.6474


# Testing on other test set

In [23]:
data_test = pd.read_csv("/kaggle/input/finance-corpus/FinancialPhraseBank_test.csv")

In [24]:
X_test, doc_spacy_data_test, Y_test = autonlp.preprocess_test_data(data_test)

- Apply small clean of texts...
- Apply nlp.pipe from spacy...
- Apply entities preprocessing...


In [25]:
name_logs = 'best_logs'
on_test_data = False
autonlp.leader_predict(name_logs = name_logs, on_test_data = on_test_data, x = X_test, y=Y_test,
                       doc_spacy_data_test = doc_spacy_data_test)


tf+Logistic_Regression_ADJ_NOUN_VERB_lem Model:
Model fold2.joblib:
Inference : 969 samples in 0.029127836227416992 sec ( 33267.146671468676 / sample).

Scores :
accuracy = 0.7389
precision macro = 0.6945
recall macro = 0.6453
f1 score macro = 0.665


Model fold3.joblib:
Inference : 969 samples in 0.028990507125854492 sec ( 33424.73437230149 / sample).

Scores :
accuracy = 0.7307
precision macro = 0.6916
recall macro = 0.6398
f1 score macro = 0.6604


Model fold0.joblib:
Inference : 969 samples in 0.02864384651184082 sec ( 33829.25542487577 / sample).

Scores :
accuracy = 0.7492
precision macro = 0.7082
recall macro = 0.6546
f1 score macro = 0.6759


Model fold4.joblib:
Inference : 969 samples in 0.02941107749938965 sec ( 32946.76980196013 / sample).

Scores :
accuracy = 0.7399
precision macro = 0.7016
recall macro = 0.6459
f1 score macro = 0.6678


Model fold1.joblib:
Inference : 969 samples in 0.02834153175354004 sec ( 34190.10688718212 / sample).

Scores :
accuracy = 0.742
precisio

In [26]:
leaderboard_test = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='test', info_models=autonlp.info_models)
print('\nTest Leaderboard')
leaderboard_test


Test Leaderboard


,name,accuracy_test,recall_macro_test,precision_macro_test,f1_macro_test
2,tf-idf+SGD_Classifier_ADJ_NOUN_VERB_lem,0.7410,0.6616,0.7121,0.6818
0,tf+Logistic_Regression_ADJ_NOUN_VERB_lem,0.7420,0.6468,0.7025,0.6686
3,BlendModel,0.7368,0.6510,0.6857,0.6645
1,tf+Naive_Bayes_ADJ_NOUN_VERB_lem,0.7069,0.6477,0.6474,0.6468
